In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 20160510: both versions of BkAvgAst created (nohist & ultcert)
###           used Tickers from earlier version, then supplemented with missing Tickers if aeTotAst>800bn
### 20170407: updated for 2016-Dec updated FailedBank list and 2017-Mar institutions2.csv file
### 20180827: updated for 2018-Jun FailedBank list and 2018-Aug institutions2.csv file

import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201806.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2018-06-30  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2018-06-30      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2018-06-30           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2018-06-30     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.461295e+09  1.435970e+09  5.103033e+08  6.666457e+08   
1 2001-03-31  1.255158e+09  1.259335e+09  6.572162e+08  7.949126e+08   
2 2001-03-31  1.053310e+09  1.050354e+09  5.043212e+08  2.793764e+08   
3 2001-03-31  8.702706e+08  8.583451e+08  5.267557e+08  6.019828e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.535391e+08  9.201849e+08        NaT  9.193848   9.193848  5.638326   
1  1.059607e+08  9.008733e+08 2009-01-16  7.896924  17.090772  7.261562   
2  4.392890e+08  7.186653e+08 2008-11-23  6.626980  23.717752  5.572229   
3  6.760383e+07  6.695866e+08        NaT  5.475375  29.193127  5.820108   
4  2.469325e+07  2.951088e+08        NaT  2.778653  31.971779  2.813476   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.638326  7.365749   7.365749  
1  12.899888  8.782965  16.148713  
2  18.472117  3.086821  19.235534  
3  24.292225  6.651290  25.886824  
4  27.105701  2.987814  28.874638

In [3]:
len(rk) # 10772

10772

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_201808.csv", encoding='latin-1')
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4     0.0     0.0       22 Main Street     48,570      NM   
1  Connecticut     6     0.0     0.0  81 West Main Street    624,655      NM   
2        Maine     8     0.0     0.0      One City Center  1,699,404      SM   
3        Maine     9  9781.0     0.0       66 Main Street    539,169      SM   
4        Maine    10     0.0     0.0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0     223.0       0.0       0.0 ...  Hartford-West Hartford-East Hartford, CT   
1     223.0     999.0       0.0 ...                     New Haven-Milford, CT   
2     223.0       0.0       0.0 ...               Portland-South Portland, ME   
3     223.0       0.0       0.0 ...                                       NaN   
4     223.0       0.0       0.0 ...               Portland-South Portland, ME   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0  25540.0  Hartford-West Hartford-East Hartford, CT    25540.0   
1  35300.0                     New Haven-Milford, CT    35300.0   
2  38860.0               Portland-South Portland, ME    38860.0   
3      0.0                                       NaN        0.0   
4  38860.0               Portland-South Portland, ME    38860.0   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0            1.0            0.0       NaN          0.0          0.0 NaN  
1            1.0            0.0       NaN          0.0          0.0 NaN  
2            1.0            0.0       NaN          0.0          0.0 NaN  
3            0.0            0.0       NaN          0.0          0.0 NaN  
4            1.0            0.0       NaN          0.0          0.0 NaN  

[5 rows x 133 columns]

In [5]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [6]:
len(bhc) # 27607

27607

In [7]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4     0.0     0.0       22 Main Street   48,570      NM   
1  Connecticut     6     0.0     0.0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0     223.0       0.0       0.0 ...  Hartford-West Hartford-East Hartford, CT   
1     223.0     999.0       0.0 ...                     New Haven-Milford, CT   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0  25540.0  Hartford-West Hartford-East Hartford, CT    25540.0   
1  35300.0                     New Haven-Milford, CT    35300.0   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0            1.0            0.0       NaN          0.0          0.0 NaN  
1            1.0            0.0       NaN          0.0          0.0 NaN  

[2 rows x 133 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
#bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
bhc2.rename(index=str, columns={"CERT":"Cert", "FED_RSSD":"IDRSSD", "NAME":"Name", "STALP":"ST", "BKCLASS":"BkClass", \
            "OFFICES":"Offices", "NAMEHCR":"Parent", "RSSDHCR":"ParRSSD", "STALPHCR":"ParST", "ULTCERT":"UltCert"}, inplace=True)
len(bhc2) # 27607

27607

In [9]:
bhc2.head(2)

Cert    IDRSSD                                    Name  ST BkClass  \
0     4  573401.0  The Southington Bank and Trust Company  CT      NM   
1     6  148304.0              Colonial Bank of Waterbury  CT      NM   

   Offices Parent  ParRSSD ParST  UltCert  
0      4.0    NaN      NaN   NaN   3510.0  
1     34.0    NaN      NaN   NaN   3510.0

In [10]:
bhc2.columns

Index([u'Cert', u'IDRSSD', u'Name', u'ST', u'BkClass', u'Offices', u'Parent',
       u'ParRSSD', u'ParST', u'UltCert'],
      dtype='object')

In [11]:
bhc2[bhc2['Parent']>' '][0:10]

Cert    IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301.0  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635.0                           AuburnBank  AL      SM      8.0   
19    39  430036.0            Robertson Banking Company  AL      NM      7.0   
21    41  432432.0                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937.0            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336.0               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032.0                Community Spirit Bank  AL      NM      5.0   
30    51  382537.0                   The Bank of Vernon  AL      NM      3.0   
31    52  293437.0        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537.0         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA     14.0  
15        Auburn National Bancorp Inc  1129533.0    AL     35.0  
19                            Rbc Inc  2526755.0    AL     39.0  
21       Phenix-Girard Bancshares Inc  2013246.0    AL     41.0  
25         Merchants Fin Services Inc  1082955.0    AL     46.0  
26   Traders & Farmers Bancshares Inc  3467305.0    AL     47.0  
29  Independent Bancshares IncESOPlan  3587294.0    AL     50.0  
30        First Vernon Bancshares Inc  1139998.0    AL     51.0  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL     52.0  
33                       Gadsden Corp  1079067.0    AL     54.0

In [12]:
#bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

In [13]:
#bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

In [14]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201712.csv", index_col=0)
#rkhn = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612n.csv", index_col=0)

#len(rkh), len(rkhn) # 10762
len(rkh) # 10767

10767

In [15]:
len(rkh[rkh['Ticker']>'']) # count before new BHCs added: 3049

3045

In [16]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 8714

8714

In [17]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10763  10764  1094314.0    MO  CBCYB
10766  10767  3447567.0    TX    NaN

In [18]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 411

410

In [19]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [20]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [21]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACFC          1
ACNB          1

In [22]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [23]:
rkh[rkh['Ticker']=="BHWB"].iloc[:,0:12] # check on Ticker that was misidentified in earlier runs (should be 2 banks only)

Rank  IDRSSD       xAsOf           xName xState  xCert      nAsOf  \
1740  1741   27548  12/31/2017  BLACKHAWK BANK     WI  14078  3/31/2001   
7693  7694  388931   9/30/2003   FIRST BANK BC     IL  16551  3/31/2001   

     FailDate BkClass  Offices                 Parent    ParRSSD  
1740      NaN      NM      9.0  Blackhawk Bancorp Inc  1491913.0  
7693      NaN      NM      4.0  Blackhawk Bancorp Inc  1491913.0

In [24]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert    IDRSSD                                    Name  ST BkClass  \
0     4  573401.0  The Southington Bank and Trust Company  CT      NM   
1     6  148304.0              Colonial Bank of Waterbury  CT      NM   
2     8  422406.0                     Fleet Bank of Maine  ME      SM   
3     9  563907.0                     Union Trust Company  ME      SM   
4    10  112109.0               Northeast Bank of Sanford  ME      SM   

   Offices Parent  ParRSSD ParST  UltCert Ticker  
0      4.0    NaN      NaN   NaN   3510.0    NaN  
1     34.0    NaN      NaN   NaN   3510.0    NaN  
2     55.0    NaN      NaN   NaN   3510.0    NaN  
3     13.0    NaN      NaN   NaN   4255.0    NaN  
4      5.0    NaN      NaN   NaN   3510.0    NaN

In [25]:
bhc3[bhc3['Cert']==3510]

Cert    IDRSSD                                   Name  ST BkClass  \
2233  3510  480228.0  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4488.0  Bank Of America Corp  1073757.0    NC   3510.0    BAC

In [26]:
bhc2[bhc2['Cert']==3510]

Cert    IDRSSD                                   Name  ST BkClass  \
2233  3510  480228.0  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4488.0  Bank Of America Corp  1073757.0    NC   3510.0

In [27]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2018-06-30  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2018-06-30      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eTLcs    pr_eDD  \
0  1.461295e+09  1.435970e+09  5.103033e+08   ...      5.638326  7.365749   
1  1.255158e+09  1.259335e+09  6.572162e+08   ...     12.899888  8.782965   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.365749   628.0        N   5202.0  Jpmorgan Chase & Co.  1039502.0   
1  16.148713  3510.0        N   4488.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [28]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [29]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2018-06-30  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2018-06-30      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eDD   pr_eDDcs  \
0  1.461295e+09  1.435970e+09  5.103033e+08   ...    7.365749   7.365749   
1  1.255158e+09  1.259335e+09  6.572162e+08   ...    8.782965  16.148713   

     Cert BkClass  Offices                Parent    ParRSSD  ParST  UltCert  \
0   628.0       N   5202.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0   
1  3510.0       N   4488.0  Bank Of America Corp  1073757.0     NC   3510.0   

   Ticker  
0     JPM  
1     BAC  

[2 rows x 28 columns]

In [30]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [31]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2018-06-30  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2018-06-30      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices    ...           aeTotLn        aeDDep  \
0        NaT       N   5202.0    ...      5.103033e+08  6.666457e+08   
1 2009-01-16       N   4488.0    ...      6.572162e+08  7.949126e+08   

         aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
0  2.535391e+08  9.201849e+08  9.193848   9.193848  5.638326   5.638326   
1  1.059607e+08  9.008733e+08  7.896924  17.090772  7.261562  12.899888   

     pr_eDD   pr_eDDcs  
0  7.365749   7.365749  
1  8.782965  16.148713  

[2 rows x 27 columns]

In [32]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201806.csv")

In [33]:
len(rk4) # 10772

10772

In [34]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 200

200

In [35]:
rkm.head()

Rank  IDRSSD      xAsOf                         xName xState    xCert  \
143   144  527954 2018-06-30  FIRST NATIONAL BANK OF OMAHA     NE   5452.0   
150   151  311845 2018-06-30                   ARVEST BANK     AR   8728.0   
172   173  137915 2018-06-30                 EMIGRANT BANK     NY  12054.0   
180   181  877369 2018-06-30                  RABOBANK, NA     CA  23364.0   
185   186  249612 2018-06-30        APPLE BANK FOR SAVINGS     NY  16068.0   

         nAsOf FailDate BkClass  Offices    ...           aeTotLn  \
143 2001-03-31      NaT       N    128.0    ...      7.707800e+06   
150 2001-03-31      NaT      SM    303.0    ...      6.812231e+06   
172 2001-03-31      NaT      SB      5.0    ...      5.135866e+06   
180 2001-03-31      NaT       N    107.0    ...      6.058033e+06   
185 2001-03-31      NaT      SB     80.0    ...      4.294454e+06   

           aeDDep       aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
143  9.046705e+06     0.000000  9.046705e+06  0.071484  73.870389  0.085163   
150  9.067994e+06  3287.771429  9.071282e+06  0.067666  74.357734  0.075268   
172  6.256872e+06     0.000000  6.256872e+06  0.056603  75.691655  0.056746   
180  6.068418e+06     0.000000  6.068418e+06  0.054708  76.133031  0.066935   
185  7.406023e+06     0.000000  7.406023e+06  0.054020  76.403840  0.047449   

      pr_eTLcs    pr_eDD   pr_eDDcs  
143  69.914601  0.099957  73.577583  
150  70.489726  0.100192  74.142313  
172  71.914724  0.069132  75.815005  
180  72.466592  0.067050  76.282166  
185  72.768095  0.081829  76.627959  

[5 rows x 27 columns]

In [36]:
rkm.to_csv('rk_missingtickers201806.csv')

In [37]:
## start up here after filling in missing tickers

In [38]:
## filled in many missing tickers

In [39]:
rkm2 = pd.read_csv('rk_missingtickers201806a.csv',index_col=0)
rkm2.head()

Rank  IDRSSD      xAsOf                         xName xState  xCert  \
143   144  527954  6/30/2018  FIRST NATIONAL BANK OF OMAHA     NE   5452   
150   151  311845  6/30/2018                   ARVEST BANK     AR   8728   
172   173  137915  6/30/2018                 EMIGRANT BANK     NY  12054   
180   181  877369  6/30/2018                  RABOBANK, NA     CA  23364   
185   186  249612  6/30/2018        APPLE BANK FOR SAVINGS     NY  16068   

         nAsOf FailDate BkClass  Offices    ...          aeTotLn       aeDDep  \
143  3/31/2001      NaN       N      128    ...      7707799.629  9046705.014   
150  3/31/2001      NaN      SM      303    ...      6812231.357  9067993.943   
172  3/31/2001      NaN      SB        5    ...      5135866.371  6256872.086   
180  3/31/2001      NaN       N      107    ...      6058032.557  6068417.629   
185  3/31/2001      NaN      SB       80    ...      4294454.143  7406022.529   

          aeFDep     aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
143     0.000000  9046705.014  0.071484  73.870389  0.085163  69.914601   
150  3287.771429  9071281.714  0.067666  74.357734  0.075268  70.489726   
172     0.000000  6256872.086  0.056603  75.691655  0.056746  71.914724   
180     0.000000  6068417.629  0.054708  76.133031  0.066935  72.466592   
185     0.000000  7406022.529  0.054020  76.403840  0.047449  72.768095   

       pr_eDD   pr_eDDcs  
143  0.099957  73.577583  
150  0.100192  74.142313  
172  0.069132  75.815005  
180  0.067050  76.282166  
185  0.081829  76.627959  

[5 rows x 27 columns]

In [40]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
350  1238967    BNS

In [41]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 1, 1

(1, 1)

In [42]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName               Parent Ticker
350  SCOTIABANK DE PUERTO RICO  Bank Of Nova Scotia    NaN

In [43]:
rk4c = rk4.copy()

In [44]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1238967L: 'BNS'}

In [45]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [46]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName               Parent Ticker
350  SCOTIABANK DE PUERTO RICO  Bank Of Nova Scotia    BNS

In [47]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 509, 508

(509, 508)

In [48]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201806.csv")

In [49]:
## process ultcert version:

In [50]:
rk5 = rk4.copy()

In [51]:
ult = set(rk5['UltCert'])
len(ult) # 5870

5870

In [52]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [53]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4303

4303

In [54]:
len(bhcc[bhcc['Ticker']>'']) # 498

498

In [55]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [56]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
38     16571.0                 Bank Of Montreal    BMO  1231333.0     0
201     4999.0              Cadence Bancorp Llc   CADE  4037349.0    TX
256    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
476    35385.0        United Community Fin Corp   UCFC  3831250.0    OH
486    26342.0             Royal Bank Of Canada     RY  1232497.0     0
664     4999.0              Cadence Bancorp Llc   CADE  4037349.0    TX
726    35525.0  Atlantic Capital Bancshares Inc    NaN  3555686.0    GA
976    35525.0  Atlantic Capital Bancshares Inc    NaN  3555686.0    GA
1083    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1185   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1635    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
1690   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1755   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1772      89.0   White River Bancshares Company    NaN  3306589.0    AR
2033   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
2159   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2363   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
2629   26342.0             Royal Bank Of Canada     RY  1232497.0     0
3097   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4045   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4179   35385.0        United Community Fin Corp   UCFC  3831250.0    OH
4311   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4444   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
4698   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
4716   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
5140    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
5935    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6811   17399.0                   First Fin Corp    NaN  1067514.0    KS
7068      89.0   White River Bancshares Company    NaN  3306589.0    AR
7074   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
7419   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7469   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7566   15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
7727    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
8745   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
9094   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9652    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9776   17399.0                   First Fin Corp    NaN  1067514.0    KS
10155  15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
10188  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10229  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10511  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [57]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6811  17399.0  First Fin Corp    NaN  1067514.0    KS
9776  17399.0  First Fin Corp    NaN  1067514.0    KS

In [58]:
len(bhcc) # 4303

4303

In [59]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4261

4261

In [60]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [61]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [62]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [63]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

u'Jpmorgan Chase & Co.'

In [64]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [65]:
len(rk5c[rk5c['Ticker']>'']) # 3102

3102

In [66]:
j # 4401

4401

In [67]:
len(rk5c) # 10772

10772

In [68]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult201806.csv")

In [69]:
# I use ZION to show how many historic banks and mergers got collapsed into one bank charter

In [70]:
rk4[rk4['Ticker']=='ZION']  ## current active affiliate (1 bank list)

Rank  IDRSSD      xAsOf   xName xState   xCert      nAsOf FailDate  \
82    83  276579 2018-06-30  ZB, NA     UT  2270.0 2001-03-31      NaT   

   BkClass  Offices    ...           aeTotLn        aeDDep         aeFDep  \
82       N    444.0    ...      1.586749e+07  1.718029e+07  875288.271429   

        aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
82  1.805557e+07  0.143107  67.726287  0.175319  63.116334  0.189824   

     pr_eDDcs  
82  66.491115  

[1 rows x 27 columns]

In [71]:
rk5c[rk5c['Ticker']=='ZION']  ## all acquired & consolidated banks (23 banks)

Rank   IDRSSD      xAsOf                                 xName xState  \
82      83   276579 2018-06-30                                ZB, NA     UT   
153    154   837260 2015-09-30               CALIFORNIA BANK & TRUST     CA   
156    157   676656 2015-09-30                        AMEGY BANK, NA     TX   
302    303  1004368 2015-09-30              NATIONAL BANK OF ARIZONA     AZ   
330    331   456960 2015-09-30                     NEVADA STATE BANK     NV   
428    429   933957 2015-09-30              VECTRA BANK COLORADO, NA     NM   
707    708   807665 2009-06-30                     VINEYARD BANK, NA     CA   
1086  1087   458263 2006-12-31                  STOCKMEN'S BANK, THE     AZ   
1130  1131  1198641 2015-09-30  COMMERCE BANK OF WASHINGTON, NA, THE     WA   
1198  1199  2479310 2008-06-30                     SILVER STATE BANK     NV   
1612  1613   720465 2008-12-31                         ALLIANCE BANK     CA   
1679  1680   618656 2004-09-30                            KLEIN BANK     TX   
2714  2715   105950 2001-09-30          MINNEQUA BANK OF PUEBLO, THE     CO   
3178  3179   576158 2003-06-30                             MAXIMBANK     TX   
4089  4090   309664 2006-06-30                           RANCHO BANK     CA   
4168  4169   521550 2003-12-31                        LONE STAR BANK     TX   
4641  4642  2068134 2009-03-31            GREAT BASIN BANK OF NEVADA     NV   
6192  6193   626754 2001-03-31              LEAGUE CITY BANK & TRUST     TX   
6372  6373  3161993 2008-03-31                           CHOICE BANK     AZ   
6672  6673   687269 2007-06-30        INTERCONTINENTAL NATIONAL BANK     TX   
6698  6699   456670 2002-09-30                   FRONTIER STATE BANK     AZ   
8411  8412  3374186 2015-03-31          COMMERCE BANK OF OREGON, THE     OR   
9893  9894  2907448 2003-06-30               SOUTHLAND BUSINESS BANK     CA   

        xCert      nAsOf   FailDate BkClass  Offices     ...      \
82     2270.0 2001-03-31        NaT       N    444.0     ...       
153   20852.0 2001-03-31        NaT      NM     99.0     ...       
156   24107.0 2001-03-31        NaT       N     81.0     ...       
302   20626.0 2001-03-31        NaT       N     66.0     ...       
330   18113.0 2001-03-31        NaT      NM     50.0     ...       
428    2993.0 2001-03-31        NaT       N     37.0     ...       
707   23556.0 2001-03-31 2009-07-17       N     17.0     ...       
1086  23182.0 2001-03-31        NaT      NM     43.0     ...       
1130  27298.0 2001-03-31        NaT       N      2.0     ...       
1198  34194.0 2001-03-31 2008-09-05      NM     18.0     ...       
1612  23124.0 2001-03-31 2009-02-06      NM      5.0     ...       
1679  22866.0 2001-03-31        NaT      NM     27.0     ...       
2714   1773.0 2001-03-31        NaT      NM      5.0     ...       
3178   9017.0 2001-03-31        NaT      NM      9.0     ...       
4089  22934.0 2001-03-31        NaT      NM      4.0     ...       
4168  22304.0 2001-03-31        NaT      NM      6.0     ...       
4641  33824.0 2001-03-31 2009-04-17      NM      5.0     ...       
6192  17577.0 2001-03-31        NaT      NM      4.0     ...       
6372  57469.0 2003-03-31        NaT      NM      3.0     ...       
6672  19112.0 2001-03-31        NaT       N      3.0     ...       
6698  24227.0 2001-03-31        NaT      NM      7.0     ...       
8411  58223.0 2005-12-31        NaT      NM      1.0     ...       
9893  35287.0 2001-03-31        NaT      NM      1.0     ...       

           aeTotLn        aeDDep         aeFDep      aeTotDep    pr_eTA  \
82    1.586749e+07  1.718029e+07  875288.271429  1.805557e+07  0.143107   
153   7.675023e+06  8.519186e+06   32194.661017  8.551380e+06  0.064882   
156   6.722988e+06  7.001789e+06  646260.186441  7.648049e+06  0.062062   
302   3.325315e+06  3.496657e+06    2862.949153  3.499520e+06  0.026439   
330   2.461972e+06  3.219857e+06       0.000000  3.219857e+06  0.023259   
428   1.905816e+06  1.9167